In [1]:
from pyspark.sql import SparkSession

In [2]:
MAX_MEMORY="5g"
spark = SparkSession.builder.appName("taxi-fare-prediciton")\
                .config("spark.executor.memory", MAX_MEMORY)\
                .config("spark.driver.memory", MAX_MEMORY)\
                .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/27 06:14:44 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/04/27 06:14:44 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/04/27 06:14:44 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/04/27 06:14:44 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [3]:
trip_files = "/home/ithingvv34/data-engineering/spark/trips/*"

In [4]:
trips_df = spark.read.csv(f"file:///{trip_files}", inferSchema=True, header=True)

In [5]:
trips_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [6]:
trips_df.createOrReplaceTempView("trips")

In [7]:
query = """
SELECT 
    trip_distance,
    total_amount
FROM
    trips
WHERE
    total_amount < 5000
    AND total_amount > 0
    AND trip_distance > 0
    AND trip_distance < 500
    AND passenger_count < 4
    AND TO_DATE(tpep_pickup_datetime) >= '2021-01-01'
    AND TO_DATE(tpep_pickup_datetime) < '2021-08-01'
"""

In [8]:
data_df = spark.sql(query)
data_df.createOrReplaceTempView("data")

In [9]:
data_df.show()

+-------------+------------+
|trip_distance|total_amount|
+-------------+------------+
|          2.1|        11.8|
|          0.2|         4.3|
|         14.7|       51.95|
|         10.6|       36.35|
|         4.94|       24.36|
|          1.6|       14.15|
|          4.1|        17.3|
|          5.7|        21.8|
|          9.1|        28.8|
|          2.7|       18.95|
|         6.11|        24.3|
|         1.21|       10.79|
|          7.4|       33.92|
|         1.01|        10.3|
|         0.73|       12.09|
|         1.17|       12.36|
|         0.78|        9.96|
|         1.66|        12.3|
|         0.93|         9.3|
|         1.16|       11.84|
+-------------+------------+
only showing top 20 rows



In [10]:
data_df.describe().show()

+-------+------------------+------------------+
|summary|     trip_distance|      total_amount|
+-------+------------------+------------------+
|  count|          13126271|          13126271|
|   mean|2.8820783305479263|17.973117241906895|
| stddev| 3.820284175387752|12.975829282992352|
|    min|              0.01|              0.01|
|    max|             475.5|            4973.3|
+-------+------------------+------------------+



In [11]:
train_df, test_df = data_df.randomSplit([0.8, 0.2], seed=1)

In [12]:
print(train_df.count())
print(test_df.count())

10500537


2625734


In [13]:
from pyspark.ml.feature import VectorAssembler

In [14]:
vassembler = VectorAssembler(inputCols=["trip_distance"], outputCol="features")

In [15]:
vtrain_df = vassembler.transform(train_df)

In [16]:
vtrain_df.show()

+-------------+------------+--------+
|trip_distance|total_amount|features|
+-------------+------------+--------+
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
+-------------+------------+--------+
only showing top 20 rows



In [17]:
from pyspark.ml.regression import LinearRegression

In [18]:
lr = LinearRegression(
    maxIter=50,
    labelCol="total_amount",
    featuresCol="features"
)

In [19]:
model = lr.fit(vtrain_df)

23/04/27 06:16:25 WARN org.apache.spark.ml.util.Instrumentation: [9aebf09f] regParam is zero, which might cause numerical instability and overfitting.
23/04/27 06:16:50 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/04/27 06:16:50 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/04/27 06:16:50 WARN com.github.fommil.netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/04/27 06:16:50 WARN com.github.fommil.netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [20]:
vtest_df = vassembler.transform(test_df)

In [21]:
prediction = model.transform(vtest_df)

In [22]:
prediction.show()

+-------------+------------+--------+----------------+
|trip_distance|total_amount|features|      prediction|
+-------------+------------+--------+----------------+
|         0.01|         3.3|  [0.01]|9.42546886221043|
|         0.01|         3.3|  [0.01]|9.42546886221043|
|         0.01|         3.3|  [0.01]|9.42546886221043|
|         0.01|         3.3|  [0.01]|9.42546886221043|
|         0.01|         3.3|  [0.01]|9.42546886221043|
|         0.01|         3.3|  [0.01]|9.42546886221043|
|         0.01|         3.3|  [0.01]|9.42546886221043|
|         0.01|         3.3|  [0.01]|9.42546886221043|
|         0.01|         3.3|  [0.01]|9.42546886221043|
|         0.01|         3.3|  [0.01]|9.42546886221043|
|         0.01|         3.3|  [0.01]|9.42546886221043|
|         0.01|         3.3|  [0.01]|9.42546886221043|
|         0.01|         3.3|  [0.01]|9.42546886221043|
|         0.01|         3.3|  [0.01]|9.42546886221043|
|         0.01|         3.8|  [0.01]|9.42546886221043|
|         

In [23]:
model.summary.rootMeanSquaredError

6.284656849365224

In [24]:
model.summary.r2

0.7660322673703551

In [25]:
from pyspark.sql.types import DoubleType
distance_list = [1.1]
distance_df = spark.createDataFrame(distance_list, DoubleType()).toDF("trip_distance")

In [26]:
distance_df.show()

+-------------+
|trip_distance|
+-------------+
|          1.1|
+-------------+



In [27]:
vdistance_df = vassembler.transform(distance_df)

In [28]:
vdistance_df.show()

+-------------+--------+
|trip_distance|features|
+-------------+--------+
|          1.1|   [1.1]|
+-------------+--------+



In [29]:
model.transform(vdistance_df).show()

+-------------+--------+------------------+
|trip_distance|features|        prediction|
+-------------+--------+------------------+
|          1.1|   [1.1]|12.669472738013734|
+-------------+--------+------------------+

